In [33]:
from transformers import pipeline
from transformers import BertForMaskedLM
from transformers import BertTokenizer
import torch.nn
import torch
from torch.nn import functional as F
from transformers import RepetitionPenaltyLogitsProcessor
import torch.nn as nn

In [34]:
# RUN THIS TO GET ~6HR TRAINED V2 MODEL WITH W2V EMBEDDINGS
MODEL = './polished/models/v2bert/bert_model/'
TOKENIZER = './polished/models/v2bert/berttokenizer/'

In [35]:
# RUN THIS TO GET ~2HR TRAINED V2 MODEL WITHOUT W2V EMBEDDINGS
MODEL = './polished/models/ka_only_no_w2v_bert/ka_only_no_w2v_bert_model//'
TOKENIZER = './polished/models/v2bert/berttokenizer/' # tokenizer same

In [36]:
# RUN THIS TO GET 10HR TRAINED V1 MODEL
MODEL = './polished/models/bert/model/'
TOKENIZER = './polished/models/bert/berttokenizer/'

In [37]:
model = BertForMaskedLM.from_pretrained(MODEL)
tokenizer = BertTokenizer.from_pretrained (TOKENIZER)

Some weights of the model checkpoint at ./polished/models/bert/model/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
tokenizer('რა მაგარია')

{'input_ids': [2, 1437, 13146, 3], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [39]:
repetition_penalty=True

In [40]:
def infer_next_token_softmax(token_ids: list[int]): # we are inferring next token. last token should not be [SEP]
    assert token_ids[-1] != tokenizer.sep_token_id
    i = len(token_ids)
    token_ids = token_ids + [tokenizer.mask_token_id] # + [tokenizer.pad_token_id]*(50-len(token_ids)-10) + [tokenizer.sep_token_id] + [tokenizer.sep_token_id]
    token_ids = torch.tensor(token_ids).view((1, -1))
    #attention_mask = (token_ids != tokenizer.pad_token_id)*1
    #print(attention_mask)
    next_token_scores = model(input_ids=token_ids).logits[0, i, :] 
    logits = F.softmax(next_token_scores, dim=0)
    #print(logits.shape)
    if repetition_penalty:
        logits = RepetitionPenaltyLogitsProcessor(10.0)(token_ids.view(1, -1), logits.view(1, -1))
    return logits.view(-1)

In [10]:
scores = infer_next_token_softmax(tokenizer('რა მაგარია')['input_ids'][:-1])

In [11]:
scores

tensor([1.6969e-12, 4.7368e-06, 7.0409e-07,  ..., 5.2675e-07, 5.0642e-07,
        5.2823e-07], grad_fn=<ViewBackward0>)

In [41]:
def get_perplexity(sentence: str):
    token_ids = tokenizer(sentence)['input_ids']
    perp = 1
    for i in range(1, len(token_ids) - 1):
        cur_context = token_ids[:i]
        expected_token = token_ids[i]
        perp = perp + torch.log(infer_next_token_softmax(cur_context)[expected_token])
    return float(torch.exp(-perp/(len(token_ids)-2)))

In [18]:
perplexities = []
for line in open('./no_en_data/ka_nse_test.txt').readlines()[:200]:
    if len(line.strip()) < 5:
        continue
    perplexities.append((get_perplexity(line), line))

In [19]:
print(f'avg perplexity = {sum([p for p, _ in perplexities])/len(perplexities)}')

avg perplexity = 2886.7350207955496


In [20]:
perplexities = sorted(perplexities)

In [21]:
# top perplexities
perplexities[:10]

[(84.50183868408203, 'შენ… – დიახ, მე ეს ვიცი!\n'),
 (223.10389709472656, 'ეს შესაძლებელი იყო.\n'),
 (226.61878967285156, 'მართლა ახალგაზრდა და ლამაზი იყო.\n'),
 (264.1933288574219, '-მაშ, როდის შეიძლება შევხვდეთ ერთმანეთს?\n'),
 (274.5380554199219, 'არა, ყინული არ მინდა.\n'),
 (368.1506042480469, 'შემეძლო ისევ შემყვარებოდა.\n'),
 (450.12408447265625,
  '-ისიც, ვინც მე მიყვარდა,ხშირად მიემგზავრებოდა ხოლმე.\n'),
 (462.02880859375,
  'მაგრამ შენ არ გახსოვს, ვერც კი ხვდები ამას, ჩემო საყვარელო!\n'),
 (473.8302307128906, '“თანახმა ვარ," ვთქვი მე, “ წავიდეთ".\n'),
 (486.3359680175781, 'მივხვდი, სადაც მოვხვდი.\n')]

In [22]:
# bad perplexities
perplexities[-10:]

[(7869.26025390625,
  'ორმოცი საათის განმავლობაში ვიჯექი მის ლოგინთან, სანამ საბრალო ბავშვის სხეულს გრიპისაგან ცხელების ალმური ასდიოდა.\n'),
 (7888.2626953125, 'დღეს "აღიარებას“ ვარქმევ ამას.\n'),
 (7888.2626953125, 'დღეს "აღიარებას“ ვარქმევ ამას.\n'),
 (8620.7041015625, 'დღეს "მოწიფულობა“ დავარქვი ამას.\n'),
 (8620.7041015625, 'დღეს "მოწიფულობა“ დავარქვი ამას.\n'),
 (9148.73828125,
  'გასეირნების შემდეგ მე და ჩემი ამხანაგი გოგონა სადარბაზო კართან ვიდექით და ვლაპარაკობდით.\n'),
 (10736.5009765625, 'განთიადს ხელი ვტაცე.\n'),
 (12525.2041015625,
  'გამომშვიდობებისას კვლავ მაჩუქე რამდენიმე ვარდი-გამომშვიდობებისას!\n'),
 (14408.65625, '“ჯორჯი რუთს მკერდზე კოცნიდა.\n'),
 (22344.73828125, 'ღვინო უმისოდაც მშვენიერია".\n')]

In [231]:
get_perplexity('ჩემი აზრით ეს არის ძალიან კარგი იდეა')

602.6224365234375

In [232]:
get_perplexity('ჩემი ძალიან ეს არის აზრით იდეა კარგი')

582.3827514648438

In [233]:
get_perplexity('ჩემი ჩემი ჩემი ჩემი ჩემი ჩემი ჩემი')

829.9646606445312

In [234]:
get_perplexity('ჩემი და რომ მაშინ რადგან რისთვის იმიტომ')

641.9747924804688

In [42]:
def get_next_top_k(token_ids: list[int], k: int):
    probs = infer_next_token_softmax(token_ids)
    tops = list(reversed(sorted([(float(v), i) for i, v in enumerate(probs)])))[:k]
    return [(p, i) for p, i in tops] # p, token_id

In [43]:
def beam_search(sentence: str, num_tokens: int, k: int):
    assert num_tokens >= 1
    token_ids = tokenizer(sentence)['input_ids'][:-1]
    cur = [(1.0, [])]
    for _ in range(num_tokens):
        nexts = []
        for p, i in cur:
            nexts += [(c_p*p, i+[j]) for c_p, j in get_next_top_k(token_ids + i, k)]
        cur = list(reversed(sorted(nexts)))[:k]
    print([tokenizer.decode(token_ids[1:]+toks) for _, toks in cur])

In [44]:
get_next_top_k(tokenizer('პრეზიდენტი მიხეილ')['input_ids'][:-1], 20)

[(0.022892849519848824, 7810),
 (0.017092948779463768, 5881),
 (0.01578742265701294, 11108),
 (0.013783823698759079, 5596),
 (0.012934033758938313, 6519),
 (0.009565254673361778, 5129),
 (0.009293988347053528, 13018),
 (0.009212212637066841, 2386),
 (0.008399723097682, 5837),
 (0.007713967934250832, 4153),
 (0.007616074755787849, 8524),
 (0.007227797992527485, 16),
 (0.006876859813928604, 2867),
 (0.006577273830771446, 5347),
 (0.006285976618528366, 2236),
 (0.005495554301887751, 4073),
 (0.0054209004156291485, 17),
 (0.005163147114217281, 6097),
 (0.005150674842298031, 5918),
 (0.004721011035144329, 4680)]

In [45]:
tokenizer.ids_to_tokens[6648]

'მმართ'

In [46]:
beam_search('1+1=', 1, 5)

['1 + 1 = 2', '1 + 1 = /', '1 + 1 = 3', '1 + 1 = 4', '1 + 1 = 1']


In [47]:
beam_search('1+1=', 3, 5)

['1 + 1 = 2 - 3', '1 + 1 = 2, 4', '1 + 1 = 2, 3', '1 + 1 = 3, 2', '1 + 1 = 2 - 4']


In [48]:
beam_search('პრეზიდენტი მიხეილ', 1, 5)

['პრეზიდენტი მიხეილ ივანიშვილის', 'პრეზიდენტი მიხეილ ბიძინა', 'პრეზიდენტი მიხეილ ზურაბიშვილი', 'პრეზიდენტი მიხეილ სააკაშვილის', 'პრეზიდენტი მიხეილ სააკაშვილი']


In [49]:
beam_search('პრეზიდენტი მიხეილ სააკ', 1, 5)

['პრეზიდენტი მიხეილ სააკაშვილი', 'პრეზიდენტი მიხეილ სააკაშვილის', 'პრეზიდენტი მიხეილ სააკაძე', 'პრეზიდენტი მიხეილ სააკაძის', 'პრეზიდენტი მიხეილ სააკიძის']


In [50]:
beam_search('პრეზიდენტი მიხეილ', 5, 5)

['პრეზიდენტი მიხეილ ზურაბიშვილი : გიორგი ვაშაძე,', 'პრეზიდენტი მიხეილ სააკაშვილი, რომ განაცხადა -', 'პრეზიდენტი მიხეილ სააკაშვილი, რომ აღნიშნა -', 'პრეზიდენტი მიხეილ სააკაშვილი, რომ ამბობს,', 'პრეზიდენტი მიხეილ ზურაბიშვილი : გიორგი ვაშაძე -']


### Masked Language Modelling

In [206]:
from transformers import pipeline

In [207]:
fill = pipeline('fill-mask', model=MODEL, tokenizer=TOKENIZER)

Some weights of the model checkpoint at ./polished/models/v2bert/bert_model/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [208]:
fill('პრეზიდენტი მიხეილ [MASK] აზრით ამ შენობის აშენება კარგი იდეაა.')

[{'score': 0.15355493128299713,
  'token': 5,
  'token_str': '!',
  'sequence': 'პრეზიდენტი მიხეილ! აზრით ამ შენობის აშენება კარგი იდეაა.'},
 {'score': 0.08936669677495956,
  'token': 720,
  'token_str': '##ის',
  'sequence': 'პრეზიდენტი მიხეილის აზრით ამ შენობის აშენება კარგი იდეაა.'},
 {'score': 0.08673388510942459,
  'token': 9245,
  'token_str': 'საათამდე',
  'sequence': 'პრეზიდენტი მიხეილ საათამდე აზრით ამ შენობის აშენება კარგი იდეაა.'},
 {'score': 0.03182349354028702,
  'token': 4499,
  'token_str': 'გამყიდველისაგან',
  'sequence': 'პრეზიდენტი მიხეილ გამყიდველისაგან აზრით ამ შენობის აშენება კარგი იდეაა.'},
 {'score': 0.028807684779167175,
  'token': 747,
  'token_str': '##აც',
  'sequence': 'პრეზიდენტი მიხეილაც აზრით ამ შენობის აშენება კარგი იდეაა.'}]

In [209]:
fill('საქართველოს საუკეთესო კერძი არის [MASK], ცომში გახვეული ხორცი.')

[{'score': 0.280554860830307,
  'token': 1916,
  'token_str': '##შვილი',
  'sequence': 'საქართველოს საუკეთესო კერძი არისშვილი, ცომში გახვეული ხორცი.'},
 {'score': 0.12002026289701462,
  'token': 4996,
  'token_str': 'გურამ',
  'sequence': 'საქართველოს საუკეთესო კერძი არის გურამ, ცომში გახვეული ხორცი.'},
 {'score': 0.10072647035121918,
  'token': 2435,
  'token_str': '##იძე',
  'sequence': 'საქართველოს საუკეთესო კერძი არისიძე, ცომში გახვეული ხორცი.'},
 {'score': 0.06962481886148453,
  'token': 4238,
  'token_str': 'ილია',
  'sequence': 'საქართველოს საუკეთესო კერძი არის ილია, ცომში გახვეული ხორცი.'},
 {'score': 0.03749911114573479,
  'token': 1683,
  'token_str': 'ხან',
  'sequence': 'საქართველოს საუკეთესო კერძი არის ხან, ცომში გახვეული ხორცი.'}]

In [210]:
fill('საუკეთესო [MASK] კერძი არის ხინკალი, ცომში გახვეული ხორცი.')

[{'score': 0.14893898367881775,
  'token': 1916,
  'token_str': '##შვილი',
  'sequence': 'საუკეთესოშვილი კერძი არის ხინკალი, ცომში გახვეული ხორცი.'},
 {'score': 0.10062244534492493,
  'token': 6648,
  'token_str': '1924',
  'sequence': 'საუკეთესო 1924 კერძი არის ხინკალი, ცომში გახვეული ხორცი.'},
 {'score': 0.053386539220809937,
  'token': 2435,
  'token_str': '##იძე',
  'sequence': 'საუკეთესოიძე კერძი არის ხინკალი, ცომში გახვეული ხორცი.'},
 {'score': 0.048771098256111145,
  'token': 10863,
  'token_str': 'მაზრა',
  'sequence': 'საუკეთესო მაზრა კერძი არის ხინკალი, ცომში გახვეული ხორცი.'},
 {'score': 0.048658207058906555,
  'token': 520,
  'token_str': '##ს',
  'sequence': 'საუკეთესოს კერძი არის ხინკალი, ცომში გახვეული ხორცი.'}]